In [25]:
%%cython 
import numpy as np
from sage.matrix.matrix_integer_dense cimport Matrix_integer_dense
from sage.matrix.matrix_integer_sparse cimport Matrix_integer_sparse
# from sage.rings.integer_ring cimport IntegerRing_class
from sage.matrix.matrix2 import Matrix
from sage.rings.integer_ring import ZZ
from sage.matrix.matrix_space import MatrixSpace
# M = Matrix(ZZ, 100, 1009)
cdef long i,j, m,n
n = 100
m= 100
matspace = MatrixSpace(ZZ, n,m, False, None)
elems = np.arange(n*m,dtype=np.int_).reshape(n,n)
M = Matrix_integer_sparse(matspace)

# cdef Matrix_integer_sparse M = Matrix_integer_sparse(matspace, elems)
def fast_mat_cy(Matrix_integer_sparse M,Py_ssize_t n,Py_ssize_t m):
    cdef Py_ssize_t i,j
    for i in range(n):
        for j in range(m):
            M[i,j]= i*j
    return M

In [27]:
%%cython
    from sage.rings.integer cimport Integer
    from sage.rings.integer_ring import ZZ
    from sage.matrix.matrix_integer_dense cimport Matrix_integer_dense
    from sage.matrix.matrix_space import MatrixSpace

    def func(Matrix_integer_dense M):
        cdef Py_ssize_t i,j
        cdef Integer k 
        mat_space = MatrixSpace(ZZ, M.nrows(), M.ncols(), False, None)
        cdef Matrix_integer_dense new_mat = Matrix_integer_dense(mat_space)
        for i in range(M.nrows()):
            for j in range(M.ncols()):
                k = M[i,j]*M[j,i]
                new_mat[i,j] = k
        return new_mat

In [22]:
func()

5

In [5]:
from sage.matrix.matrix_integer_dense import Matrix_integer_dense
from sage.matrix.matrix_integer_sparse import Matrix_integer_sparse
n = 1000
m= 1000
matspace = MatrixSpace(ZZ, n,m, True, None)
elems = np.arange(n*m,dtype=np.int_).reshape(n,n)
M = Matrix_integer_sparse(matspace, elems)
def fast_mat_py(M):
    for i in range(n):
        for j in range(m):
            M[i,j] = i*j
    return M

# %timeit fast_mat_py(M)


In [7]:
%timeit fast_mat_cy(M, n, m)

190 ms ± 3.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
import numpy as np
N = 1000
n = 10000
m = 10000
entries = np.random.choice(n*m,N,replace=False)
source,target =np.unravel_index(entries, (n,m))
entries = np.random.randint(1,10,N)
rels = np.stack((source,target,entries),axis=1)

def python_rels_to_matrix(rels):
    entries_dict = {(a,b):c for a,b,c in rels}
    return Matrix(ZZ, n, m, entries_dict)
%timeit python_rels_to_matrix(rels)

2.63 ms ± 55.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
def old_method_sparse(rels):
    M = Matrix(ZZ, n, m, sparse=True)
    for a,b,c in rels:
        M[a,b] = c
    return M
def old_method_dense(rels):
    M = Matrix(ZZ, n, m, sparse=False)
    for a,b,c in rels:
        M[a,b] = c
    return M
%timeit old_method_dense(rels)
%timeit old_method_sparse(rels)

144 ms ± 824 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
2.62 ms ± 16.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
M = old_method_dense(rels)
%timeit M[:, np.arange(5)]

TypeError: only size-1 arrays can be converted to Python scalars